In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from statsmodels.graphics.tsaplots import plot_acf

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.model_selection import TimeSeriesSplit

In [3]:
# # !pip install pandas_market_calendars
# import pandas_market_calendars as mcal

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for exchange-calendars: filename=exchange_calendars-3.4-py3-none-any.whl size=179567 sha256=b93f9f4269f3edd64f77fabf9739e0a6dc973cbaeb02a7c3244be11a1464b834
  Stored in directory: c:\users\joel\appdata\local\pip\cache\wheels\7c\ba\85\1819a0f65360ce0276efe87bc37a7c0681bcbedde4582c7ce0
Successfully built exchange-calendars
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\joel\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\~andas\\_libs\\algos.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'pandas_market_calendars'

In [4]:
# To Do:
# Fix dates issue for some data sources (business days)

# shift dates if they fall on a weekend/trading holiday
def get_prev_business_date(date, holidays):
    if date.isoweekday() in set((6, 7)):
        #print(f"{date.strftime('%Y-%m-%d')} falls on a weekend, moving the date back")
        date -= pd.offsets.BDay(n = date.isoweekday() % 5)
    elif date in holidays:
        print(f"{date.strftime('%Y-%m-%d')} is a NYSE trading holiday, moving the date back")
        date -= pd.offsets.BDay(n=1)  # get prev day
        # now if the new date is on a weekend, move it back 
        # according to the original rules for weekends
        if date.isoweekday() in set((6, 7)):
            date -= pd.offsets.BDay(n = date.isoweekday() % 5)

    return date

**Loading data**

In [2]:
#Load Price Data
wd = os.getcwd()

df = pd.read_excel(wd+'/Data/'+'WTI.xlsx')
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%m-%d-%Y')
df.set_index('Date', inplace=True)

In [3]:
#Load Features

xlsx_files = glob.glob(os.path.join(wd, 'Data', "*.xlsx"))

for file in xlsx_files:
    data_name = file.split("/")[-1].split('.')[0]
    data_name = data_name.replace(" ", "_")
    
    if data_name == 'WTI':
        continue
        
    df_temp = pd.read_excel(file)
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])
    df_temp['Date'] = df_temp['Date'].dt.strftime('%m-%d-%Y')
    df_temp.set_index('Date', inplace=True)
    
    df[df_temp.columns[0] + '_' + data_name] = df_temp[df_temp.columns[0]]

#backfill data for other frequencies
df.fillna(method='bfill', inplace=True)

In [4]:
df['PX_Sign'] = np.sign(df['PX_LAST'] - df['PX_LAST'].shift(-1))
df.loc[df.PX_Sign == 0, 'PX_Sign'] = 1
df.head()

c:\users\joel\appdata\local\programs\python\python37\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in sign
  result = getattr(ufunc, method)(*inputs, **kwargs)


,PX_LAST,PX_VOLUME,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Canada_daily_prod,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Cass_Freight_Rates,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\China_GDP,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\DAX,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\DowJones,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Europe_Inflation,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Europe_IndProd_exConstruction,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Germany_GDP,...,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Russia_Industrial_Production,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\Russia_daily_prod,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\SPX,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\US_GDP,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\US_Inflation,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\USDEUR,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\US_daily_prod,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\US_IndProd_Index,PX_LAST_C:\Cornell\ORIE_5471_Big_Messy_Data\Project\ORIE5741_Project\Data\WTI,PX_Sign
Date,,,,,,,,,,,,,,,,,,,,,
10-29-2021,83.57,485727.0,4121.0,1.184,4.9,15688.77,35819.56,3.4,101.9,2.5,...,6.8,10072.497,4605.38,9.6,5.4,0.8650,11230.181,100.0178,83.57,1.0
10-28-2021,82.81,491585.0,4121.0,1.184,4.9,15696.33,35730.48,3.4,101.9,2.5,...,6.8,10072.497,4596.42,9.6,5.4,0.8559,11230.181,100.0178,82.81,1.0
10-27-2021,82.66,529952.0,4121.0,1.184,4.9,15705.81,35490.69,3.4,101.9,2.5,...,6.8,10072.497,4551.68,9.6,5.4,0.8618,11230.181,100.0178,82.66,-1.0
10-26-2021,84.65,467716.0,4121.0,1.184,4.9,15757.06,35756.88,3.4,101.9,2.5,...,6.8,10072.497,4574.79,9.6,5.4,0.8623,11230.181,100.0178,84.65,1.0
10-25-2021,83.76,568725.0,4121.0,1.184,4.9,15599.23,35741.15,3.4,101.9,2.5,...,6.8,10072.497,4566.48,9.6,5.4,0.8613,11230.181,100.0178,83.76,1.0


In [ ]:
df_t = df.copy()
ax = df_t[df_t.columns[0]][::-1].plot()
ax.set_ylabel("WTI Price")
ax.set_title('WTI Price (1990-2021)')

**Backfill data for Economic Variables appropriately**

In [ ]:
prediction = ['PX_LAST']
classification = ['PX_Sign']

In [ ]:
#Dict for frequency adjustment
cols_freq = {
    'PX_LAST':"daily",
    'PX_VOLUME':"daily",
    'PX_LAST_DowJones':"daily",
    'PX_MID_GT10':"daily",
    'PX_LAST_HangSeng':"daily",
    'PX_LAST_Nikkei':"daily",
    'PX_LAST_SPX':"daily",
    'PX_LAST_USDEUR':"daily",
    'PX_LAST_NG1':"daily",
    'PX_LAST_DAX':"daily",
    'PX_LAST_US_daily_prod':"monthly",
    'PX_LAST_Cass_Freight_Rates':"monthly",
    'PX_LAST_Canada_daily_prod':"monthly",
    'PX_LAST_Russia_daily_prod':"monthly",
    'PX_LAST_OPEC_daily_prod':"monthly",
    'PX_LAST_US_Inflation':"monthly",
    'PX_LAST_Europe_Inflation':"monthly",
    'PX_LAST_Europe_IndProd_exConstruction':"monthly",
    'PX_LAST_Russia_Industrial_Production':"monthly",
    'PX_LAST_US_IndProd_Index':"monthly",
    'PX_LAST_China_GDP':"quarterly",
    'PX_LAST_Germany_GDP':"quarterly",
    'PX_LAST_Japan_GDP':"quarterly",
    'PX_LAST_Russia_GDP':"quarterly",
    'PX_LAST_US_GDP':"quarterly",
    'PX_Sign':'daily',
}

In [ ]:
def ma(df, column, freq = "monthly"):
    if freq == "monthly":
        df[df.columns[column] + '_SMA22'] = df[df.columns[column]]
        for i in range(0, len(df[df.columns[column]])-22):
            df.iloc[i, -1] = np.mean(df.iloc[i:i+22, -2])
    elif freq == "quarterly":
        df[df.columns[column] + '_SMA65'] = df[df.columns[column]]
        for i in range(0, len(df[df.columns[column]])-65):
            df.iloc[i, -1] = np.mean(df.iloc[i:i+65, -2])
    return df

def daily_lag(df, column, lag):
    lag = lag+1
    for i in range(1, lag):
        df[df.columns[column] + '_t-' + str(i)] = df[df.columns[column]].shift(-i)
    
    return df

def lag_data(df, column, freq = "monthly"):
    if freq == "monthly":
        df[df.columns[column] + '_M'] = df[df.columns[column]].shift(-22)
    elif freq == "quarterly":
        df[df.columns[column] + '_Q'] = df[df.columns[column]].shift(-65)
    elif freq == "weekly":
        df[df.columns[column] + '_W'] = df[df.columns[column]].shift(-10)
    else:
        raise ValueError("Frequency invalid")
        
    return df

In [ ]:
def get_lag_ma_data(df, daily_lags = 1, cols_freq=cols_freq):
    for i in range(len(df.columns)):
        column_name = df.columns[i]
        
        if cols_freq[column_name] == 'daily':
            df = daily_lag(df, i, daily_lags)
        else:
            df = lag_data(df, i, cols_freq[column_name])
            df = ma(df, i, cols_freq[column_name])
    return df

**Note** - figure out what the warnings below are

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def get_final_data(df, features, target=classification):
    if target == classification:
        features = ['PX_LAST'] + features

    n = len(target + features)
    df_trial = get_lag_ma_data(df[target + features]).dropna()
    
    df_target = df_trial[target]
    df_features = df_trial.iloc[:, n:]
    
    return df_target, df_features

In [ ]:
def get_tr_val_test(df, features, target, scale=True):
    train_x, test_x, train_y, test_y = train_test_split(features, 
                                                target, 
                                                test_size=0.25, 
                                                random_state=42)
    train_x, valid_x, train_y, valid_y = train_test_split(train_x, 
                                                train_y, 
                                                test_size=0.33, 
                                                random_state=42)
    
    if scale:
        scaler = StandardScaler()
        scaled_fit_train = scaler.fit(train_x)

        train_x = scaled_fit_train.transform(train_x)
        valid_x = scaled_fit_train.transform(valid_x)
        test_x = scaled_fit_train.transform(valid_x)

        train_y = np.asarray(train_y)
        valid_y = np.asarray(train_y)
        test_y = np.asarray(train_y)
    
    print('Total Dataset Size = {}'.format(len(features)))
    print('Training Dataset Size = {}'.format(len(train_x)))
    print('Validation Dataset Size = {}'.format(len(valid_x)))
    print('Test Dataset Size = {}'.format(len(test_x)))
    
    return train_x, valid_x, test_x, train_y, valid_y, test_y



In [ ]:
# Feature Categories by Type
fin_indices = ['PX_LAST_DAX', 'PX_LAST_DowJones', 'PX_MID_GT10',
           'PX_LAST_HangSeng', 'PX_LAST_Nikkei', 'PX_LAST_SPX',
           'PX_LAST_USDEUR']
nat_gas = ['PX_LAST_NG1']
gdp = ['PX_LAST_China_GDP', 'PX_LAST_Germany_GDP', 'PX_LAST_Japan_GDP',
       'PX_LAST_Russia_GDP', 'PX_LAST_US_GDP']
oil_prod = ['PX_LAST_Canada_daily_prod', 'PX_LAST_Russia_daily_prod',
            'PX_LAST_OPEC_daily_prod', 'PX_LAST_US_daily_prod']
freight = ['PX_LAST_Cass_Freight_Rates']
inflation = ['PX_LAST_US_Inflation', 'PX_LAST_Europe_Inflation']
ind_prod = ['PX_LAST_Europe_IndProd_exConstruction', 
            'PX_LAST_Russia_Industrial_Production',
            'PX_LAST_US_IndProd_Index']

In [ ]:
# Feature Categories by Date Range
feat_1992 = ['PX_LAST_DAX', 'PX_LAST_DowJones', 'PX_MID_GT10', 
             'PX_LAST_HangSeng', 'PX_LAST_Nikkei', 'PX_LAST_SPX', 
             'PX_LAST_USDEUR', 'PX_LAST_NG1', 'PX_LAST_China_GDP', 
             'PX_LAST_Germany_GDP', 'PX_LAST_US_GDP', 'PX_LAST_US_daily_prod', 
             'PX_LAST_Cass_Freight_Rates', 'PX_LAST_US_Inflation', 
             'PX_LAST_Europe_IndProd_exConstruction', 'PX_LAST_US_IndProd_Index']
feat_1996 = feat_1992 + ['PX_LAST_Japan_GDP', 'PX_LAST_Russia_GDP',
                         'PX_LAST_Canada_daily_prod', 'PX_LAST_Russia_daily_prod']
feat_2000 = feat_1996 + ['PX_LAST_OPEC_daily_prod', 'PX_LAST_Europe_Inflation']
feat_2002 = feat_2000 + ['PX_LAST_Russia_Industrial_Production']

### Classification

In [ ]:
df_main = df_t.copy()
features = feat_2002

target, features = get_final_data(df=df_main, 
                                      features=features,
                                      target=classification)
train_x, valid_x, test_x, train_y, valid_y, test_y = get_tr_val_test(df_main, features=features,
                                                                     target=target, scale=True)

In [ ]:
features

In [ ]:
# Trial, not fair work:

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
lam = np.arange(0.01, 1.01, 0.01)

In [ ]:
def svc_hinge(lam=lam, iters=10000):
    mark = 1000
    l_star = 0
    Hloss = []
    Rh = []
    
    for i in lam:
        clf = svm.LinearSVC(penalty='l2', loss='hinge', fit_intercept=True,
                            C=(1/i), max_iter=iters, 
                            random_state=42)
        model_svm = clf.fit(train_x_std, train_y)
        train_pred = model_svm.predict(valid_x_std)
        acc = np.mean(train_pred != valid_y)
        
        if (acc<mark):
            l_star = i
            mark = acc
            model = model_svm
        
        Hloss.append(acc)
        Rh.append(i)
    print('Lambda* = {}'.format(l_star))
    print('Misclassification Error Rate = {}'.format(mark))
    return Hloss, Rh, l_star, model

In [ ]:
Hloss, R_h, l_star_h, svm_model = svc_hinge(lam, 10000)

In [ ]:
# https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/
# https://medium.com/@subpath/backtesting-time-series-models-weekend-of-a-data-scientist-92079cc2c540

def backtest_model_val(X, y, n_splits=100 model_name=''):
    predictions = []
    true_values = []
    
    splits = TimeSeriesSplit(n_splits=n_splits)
    
    for train_index, val_index in splits.split(X):
        x_train = X[train_index]
        y_train = y[train_index]
        x_val = X[val_index]
        y_val = y[val_index]
        
        model = LinearRegression()
        model.fit(x_train, y_train)
        
        predictions.append(model.predict(x_val))
        true_values.append(y_val)
        
    return true_values, predictions